---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [407]:
import pandas as pd
import numpy as np
import datetime
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [427]:
def date_sorter():
    re1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    re2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    re3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    re4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    re5 = '(\d{1,2}[/-][1|2]\d{3})'
    re6 = '([1|2]\d{3})'
    
    regex_list = '{}|{}|{}|{}|{}|{}'.format(re1,re2,re3,re4,re5,re6)
    date_parse = df.str.findall(regex_list)
    
    date_parse_4=[i[0] for i in date_parse]
    date_ls = []
    for j in date_parse_4:
        for k in j:
            if len(k) > 0:
                date_ls.append(k)
    date_ls = pd.Series(date_ls)
    date_ls = [word.replace('Janaury', 'January') for word in date_ls]
    date_ls = [word.replace('Decemeber', 'December')for word in date_ls]
    date_parse_3 = pd.to_datetime(date_ls)
    date_parse_3 = pd.Series(date_parse_3)
    ans = date_parse_3.sort_values(ascending=True).index
    ans = pd.Series(ans.values)
    return ans

In [430]:
date_sorter()

(['03/25/93',
  '6/18/85',
  '7/8/71',
  '9/27/75',
  '2/6/96',
  '7/06/79',
  '5/18/78',
  '10/24/89',
  '3/7/86',
  '4/10/71',
  '5/11/85',
  '4/09/75',
  '8/01/98',
  '1/26/72',
  '5/24/1990',
  '1/25/2011',
  '4/12/82',
  '10/13/1976',
  '4/24/98',
  '5/21/77',
  '7/21/98',
  '10/21/79',
  '3/03/90',
  '2/11/76',
  '07/25/1984',
  '4-13-82',
  '9/22/89',
  '9/02/76',
  '9/12/71',
  '10/24/86',
  '03/31/1985',
  '7/20/72',
  '4/12/87',
  '06/20/91',
  '5/12/2012',
  '3/15/83',
  '2/14/73',
  '5/24/88',
  '7/27/1986',
  '1-14-81',
  '7-29-75',
  '6/24/87',
  '8/14/94',
  '4/13/2002',
  '8/16/82',
  '2/15/1998',
  '7/15/91',
  '06/12/94',
  '9/17/84',
  '2/28/75',
  '11/22/75',
  '5/24/91',
  '6/13/92',
  '7/11/71',
  '12/26/86',
  '10/11/1987',
  '3/14/95',
  '12/01/73',
  '12/5/2010',
  '08/20/1982',
  '7/24/95',
  '8/06/83',
  '02/22/92',
  '6/28/87',
  '07/29/1994',
  '08/11/78',
  '10/29/91',
  '7/6/91',
  '1/21/87',
  '11/3/1985',
  '7/04/82',
  '4-13-89',
  '7/11/77',
  '4/12/7